In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima

In [59]:
data = pd.read_csv('data_cleaned.csv')
data.head(2)

/var/folders/tx/ccgvwwd562sfhw5ybj094vxc0000gn/T/ipykernel_94153/1202436076.py:1: DtypeWarning: Columns (14,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_cleaned.csv')


,Unnamed: 0,index_row,id,program_name,description,org_name,category_name,capacity,min_age,max_age,...,wed_end_time,wed_start_time,program_price,geographic_cluster_name,program_pays_participants,program_has_scholarships,program_provides_transportation,latitude,longitude,program_provides_free_food
0,0,"4,568",78752,BBBS Community Based Mentoring (Year Round),Community Based Mentoring is the traditional B...,Big Brothers Big Sisters,academic_support,2000.0,7,15,...,NaN,NaN,Free,NEAR WEST SIDE,Not Paid,YES,YES,41.886002,-87.642403,False
1,1,"4,569",78752,BBBS Community Based Mentoring (Year Round),Community Based Mentoring is the traditional B...,Big Brothers Big Sisters,work_career,2000.0,7,15,...,NaN,NaN,Free,NEAR WEST SIDE,Not Paid,YES,YES,41.886002,-87.642403,False


In [60]:
def count_programs_by_features(data, features):
    # Extract the month and year from the start_date column
    data['start_date'] = pd.to_datetime(data['start_date'])
    data['month'] = data['start_date'].dt.month
    data['year'] = data['start_date'].dt.year
    
    features_with_month = features + ['month', 'year']
    
    program_counts = pd.DataFrame(data.groupby(features_with_month).size().reset_index(name='count'))
    
    return program_counts

In [61]:
features = ['category_name']
program_counts = count_programs_by_features(data, features)
program_counts

,category_name,month,year,count
0,academic_support,1,2020,2
1,academic_support,1,2021,30
2,academic_support,1,2022,177
3,academic_support,1,2023,457
4,academic_support,2,2021,1
...,...,...,...,...
489,work_career,10,2022,16
490,work_career,11,2021,13
491,work_career,11,2022,9
492,work_career,12,2021,7


In [62]:
program_counts['date'] = pd.to_datetime(program_counts[['year', 'month']].assign(day=1))
program_counts.reset_index(drop=True, inplace=True)
program_counts.set_index('date', inplace=True)
program_counts.head()

,category_name,month,year,count
date,,,,
2020-01-01,academic_support,1,2020,2
2021-01-01,academic_support,1,2021,30
2022-01-01,academic_support,1,2022,177
2023-01-01,academic_support,1,2023,457
2021-02-01,academic_support,2,2021,1


In [124]:
print(program_counts)

               category_name  month  year  count
date                                            
2020-01-01  academic_support      1  2020      2
2021-01-01  academic_support      1  2021     30
2022-01-01  academic_support      1  2022    177
2023-01-01  academic_support      1  2023    457
2021-02-01  academic_support      2  2021      1
...                      ...    ...   ...    ...
2022-10-01       work_career     10  2022     16
2021-11-01       work_career     11  2021     13
2022-11-01       work_career     11  2022      9
2021-12-01       work_career     12  2021      7
2022-12-01       work_career     12  2022      3

[494 rows x 4 columns]


#### Split data

In [125]:
train_size = 0.8  # Specify the desired train set size, e.g., 80% of the data
train, test = train_test_split(program_counts, train_size=train_size, shuffle=True, random_state=42)

In [65]:
y = program_counts['count']
X = program_counts.drop('count', axis=1)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

#### ARIMA

In [127]:
test

,category_name,month,year,count
date,,,,
2020-12-01,sports_wellness,12,2020,49
2021-07-01,computers,7,2021,4
2023-01-01,performance,1,2023,29
2021-05-01,helping_your_community,5,2021,2
2021-10-01,performance,10,2021,6
...,...,...,...,...
2021-07-01,academic_support,7,2021,2
2022-12-01,science,12,2022,44
2021-05-01,science,5,2021,42


In [131]:
end_date

Timestamp('2023-09-01 00:00:00')

In [150]:
arima_model = ARIMA(endog=train['count'], order=(1, 0, 1))  # Replace order values with your chosen hyperparameters
arima_model_fit = arima_model.fit()


start_date = test.index.min()
end_date = test.index.max()
predictions = arima_model_fit.predict(start=end_date, end=start_date)

/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self

In [151]:
mae = (abs(predictions - test['count'])).mean()
mae

382.85064485621774

In [68]:
model = ARIMA(y_train, order=(1, 1, 1))  # Specify the order of the ARIMA model (p, d, q)
model_fit = model.fit()

# Predict on test data
predictions = model_fit.predict(start=len(X_train), end=len(program_counts) - 1)

/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/adazhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self

,count
date,
2022-09-01,1864
2021-07-01,3
2022-07-01,6
2020-07-01,765
2021-02-01,17
...,...
2023-03-01,1
2022-03-01,8
2021-03-01,2


In [93]:
predictions=pd.DataFrame(predictions)
y_test=pd.DataFrame(y_test)

In [106]:
y_test.index = predictions.index
y_test.dtypes

count    int64
dtype: object

In [111]:
predictions = predictions.rename(columns={'predicted_mean': 'count'})

In [113]:
rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
rmse

count    618.546832
dtype: float64

In [114]:
predictions-y_test

,count
395,-1728.345975
396,130.138240
397,127.185243
398,-631.815635
399,116.184381
...,...
489,132.184381
490,125.184381
491,131.184381
492,132.184381
